In [1]:
%load_ext autoreload
%autoreload 2

import os

import pandas as pd
from pytorch_lightning import Trainer
from pl_module import CMPNNLitModel
from dataset import CMPNNDataset
from preprocessing import scaffold_split_indices
from torch_geometric.loader import DataLoader
from torch_geometric.data import Batch
from torch.utils.data import Subset

In [2]:
df_path = "./raw/SAMPL.csv"
full_ds = CMPNNDataset(root='.', csv_file=os.path.basename(df_path), atom_messages=True, force_reload=True)
print(full_ds[0].y)
train_indices, val_indices, test_indices = scaffold_split_indices(
    pd.read_csv(df_path),
    valid_ratio=0.1,
    test_ratio=0.1,
    seed=42,
)


Processing...


tensor([-11.0100])


Done!


In [3]:
mean, std = full_ds.compute_normalization(train_indices)
print(f"Mean: {mean}, Std: {std}")

Mean: -4.139922618865967, Std: 3.9406135082244873


In [4]:
full_ds.apply_normalization()

In [5]:
# 1) load once
full_ds = CMPNNDataset(root='.', csv_file='SAMPL.csv')

# 2) see raw
print(" BEFORE normalizing:", full_ds[0].y)  
# → tensor([-11.0100])

# 3) compute train‐set mean/std
mean, std = full_ds.compute_normalization(train_indices)
print(f"Fitted mean={mean:.4f}, std={std:.4f}")
# → e.g. Fitted mean=-4.1399, std=3.9406

# 4) this is the missing step!
full_ds.apply_normalization()

# 5) now it really is normalized
print(" AFTER normalizing:", full_ds[0].y)
# → tensor([(-11.01 − (−4.1399)) / 3.9406 ]) ≈ tensor([-1.75])


 BEFORE normalizing: tensor([-11.0100])
Fitted mean=-4.1399, std=3.9406
 AFTER normalizing: tensor([-1.7434])


In [6]:
# 6) create train/val/test splits
train_ds = Subset(full_ds, train_indices)
val_ds = Subset(full_ds, val_indices)
test_ds = Subset(full_ds, test_indices)

# 7) create dataloaders
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

In [7]:
full_ds.mean

-4.139922618865967

In [8]:
from torchmetrics import MeanSquaredError, MeanAbsoluteError, R2Score
# Check device
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CMPNNLitModel(
    in_node_feats=full_ds.num_atom_features,
    in_edge_feats=full_ds.num_bond_features,
    hidden_dim=300,
    num_steps=3,
    dropout_mp=0.1,
    dropout_head=0.1,
    n_tasks=full_ds.num_targets,
    metrics=[MeanSquaredError(squared=False), MeanAbsoluteError().to(device=device), R2Score().to(device=device)],
    lr=1e-4,
    weight_decay=1e-5,
    target_mean=full_ds.mean,
    target_std=full_ds.std,
)

In [9]:
from pytorch_lightning.callbacks import EarlyStopping
trainer = Trainer(
    max_epochs=100,
    accelerator="gpu",
    devices=1,
    logger=False,
    # Allow for early stopping
    callbacks=[EarlyStopping(monitor="val_meansquarederror", patience=10)],
    enable_checkpointing=False,
    enable_progress_bar=True,
)
trainer.fit(
    model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('AMD Radeon RX 6600') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | encoder   | CMPNNEncoder     | 3.6 M  | train
1 | head      | FFNHead          | 90.3 K | train
2 | metrics   | ModuleList       | 0      | train
3 | criterion | MeanSquaredError | 0      | train
-------------------------------------------------------
3.6 M     Trainable params
0         Non-trainable params
3.6 M     Total params
14.592    Total estimated model params 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/calvin/miniforge3/envs/dmpnn_rocm/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/calvin/miniforge3/envs/dmpnn_rocm/lib/python3.10/site-packages/torch/nn/modules/linear.py:125: UserWarning: Attempting to use hipBLASLt on an unsupported architecture! Overriding blas backend to hipblas (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:310.)
  return F.linear(input, self.weight, self.bias)
/home/calvin/miniforge3/envs/dmpnn_rocm/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 223. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/home/calvin/miniforge3/envs/dmpnn_rocm/lib/python3.10/site-packages/

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [10]:
for batch in test_loader:
    print(batch.num_graphs)

32
32


In [11]:
trainer.test(
    model,
    dataloaders=test_loader,
    ckpt_path=None,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/calvin/miniforge3/envs/dmpnn_rocm/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/calvin/miniforge3/envs/dmpnn_rocm/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

/home/calvin/miniforge3/envs/dmpnn_rocm/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 243. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/home/calvin/miniforge3/envs/dmpnn_rocm/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 222. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     test_mae_unscaled     │    0.6221714019775391     │
│  test_meanabsoluteerror   │    0.15796050429344177    │
│   test_meansquarederror   │     0.229555144906044     │
│     test_r2_unscaled      │     0.922885537147522     │
│       test_r2score        │    0.9156404137611389     │
│    test_rmse_unscaled     │    0.9030335545539856     │
└───────────────────────────┴───────────────────────────┘

[{'test_meansquarederror': 0.229555144906044,
  'test_meanabsoluteerror': 0.15796050429344177,
  'test_r2score': 0.9156404137611389,
  'test_rmse_unscaled': 0.9030335545539856,
  'test_mae_unscaled': 0.6221714019775391,
  'test_r2_unscaled': 0.922885537147522}]

In [12]:
trainer.predict(
    model,
    dataloaders=test_loader,
    ckpt_path=None,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/calvin/miniforge3/envs/dmpnn_rocm/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
/home/calvin/miniforge3/envs/dmpnn_rocm/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

[tensor([ -0.5099,  -3.7192,  -4.3143,   2.4583,  -6.5929,  -7.5487,  -2.1350,
         -16.0565,  -0.4944,  -7.1750,  -1.1194,  -2.2617,  -7.2051,  -3.2161,
          -1.0347,  -6.6188,  -1.5390,  -2.2581,   0.3439,   1.9330,  -5.2738,
          -4.0901,  -4.6398,  -4.4765,  -4.5184,   2.7037,  -6.4597,   1.7081,
          -1.1818,   2.4177,  -2.5245,  -4.0098]),
 tensor([ 1.8233, -2.2247, -4.3344,  2.4373, -2.1484, -0.1741,  0.7361, -3.9501,
          0.7080, -1.6817,  2.2452, -6.6005, -4.0950, -0.6585, -2.7094, -4.8813,
         -7.2410, -1.7782, -7.5608, -5.0189,  2.0105,  0.3586, -0.0619, -0.2876,
          0.2380, -2.6157, -4.6575, -1.1893,  0.9953, -1.3103,  1.0025, -6.1955])]

## Ensemble Over Multiple Random Initializations
To reduce model variance on this small dataset, train several models with different random seeds and average their predictions.

In [16]:
import numpy as np
seeds = [42, 7, 13, 21, 99]
ensemble_preds = []
# Collect true targets once
y_true = []
for batch in test_loader:
    y_true.append(batch.y.cpu().numpy())
y_true = np.concatenate(y_true)
for seed in seeds:
    torch.manual_seed(seed)
    # reinstantiate and train model
    model = CMPNNLitModel(in_node_feats=full_ds.num_atom_features, in_edge_feats=full_ds.num_bond_features,
                             target_mean=full_ds.mean, target_std=full_ds.std,
                             hidden_dim=300, num_steps=3, dropout_mp=0.1, dropout_head=0.1,
                             n_tasks=full_ds.num_targets, metrics=[MeanSquaredError(squared=False)],
                             lr=1e-4, weight_decay=1e-5, readout='gru', use_booster=True)
    trainer = Trainer(
        max_epochs=100,
        accelerator="gpu",
        devices=1,
        logger=False,
        # Allow for early stopping
        callbacks=[EarlyStopping(monitor="val_meansquarederror", patience=10)],
        enable_checkpointing=False,
        enable_progress_bar=True,
    )
    trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)
    # predict on test set
    preds = trainer.predict(model, dataloaders=test_loader)
    preds = np.concatenate([p.cpu().numpy() for p in preds])
    ensemble_preds.append(preds)
# average ensemble predictions
ensemble_preds = np.stack(ensemble_preds, axis=0).mean(axis=0)
# compute RMSE
y_true = y_true * full_ds.std + full_ds.mean
# compute RMSE
rmse = np.sqrt(((ensemble_preds - y_true) ** 2).mean())
print(f'Ensemble Test RMSE: {rmse:.3f} kcal/mol')

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | encoder   | CMPNNEncoder     | 3.6 M  | train
1 | head      | FFNHead          | 90.3 K | train
2 | metrics   | ModuleList       | 0      | train
3 | criterion | MeanSquaredError | 0      | train
-------------------------------------------------------
3.6 M     Trainable params
0         Non-trainable params
3.6 M     Total params
14.592    Total estimated model params size (MB)
37        Modules in train mode
0         Modules in eval mode

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | encoder   | CMPNNEncoder     | 3.6 M  | train
1 | head  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

Ensemble Test RMSE: 0.736 kcal/mol


In [14]:
y_true

array([ 0.8907046 , -0.07107458,  0.0126687 ,  1.7941172 , -0.53546923,
       -0.17258164,  0.2562856 , -2.3575206 ,  0.91608137, -0.61159956,
        0.68769056,  0.30703914, -0.62936324,  0.3451043 ,  0.4618374 ,
       -0.46948963,  0.41869688, -0.00509495,  1.0252014 ,  1.537812  ,
       -0.15228014,  0.04819619, -0.16750629, -0.10913975, -0.0634615 ,
        1.8042679 , -1.0176276 ,  1.4769077 ,  0.7562078 ,  1.7839665 ,
        0.38063172,  0.12686414,  1.4718324 ,  0.5379677 , -0.21064669,
        1.6545451 ,  0.52020395,  1.0023624 ,  1.1190954 , -0.09645132,
        1.0252014 ,  0.4516867 ,  1.6875348 , -0.65474   , -0.1040644 ,
        0.7536701 ,  0.2182205 , -0.40351012, -0.8526787 , -0.04316011,
       -0.40351012, -0.1801946 ,  1.6875348 ,  1.053116  ,  0.9871363 ,
        0.9338451 ,  1.192688  , -0.01524564, -0.0634615 ,  0.18015534,
        1.2307532 ,  0.5328923 ,  1.1241708 , -0.2334858 ], dtype=float32)